**The VAE implmentation we used**

trained and ready for export

In [ ]:
#!pip install pyod
#converting vae to a tf lite interpreter 
import tensorflow as tf
from pyod.models.vae import VAE
from pyod.models.auto_encoder import AutoEncoder
from pyod.utils.data import generate_data

model = VAE([1,16],[16,1],epochs=10) 
X_train, y_train = generate_data(behaviour='new', n_features=300, train_only=True)
model.fit(X_train)  

export pyod to tflite 

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model.model_)
converter.allow_custom_ops = False
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                        tf.lite.OpsSet.SELECT_TF_OPS]
converter.target_spec.supported_ops=[  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
                                      tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()
open("pyod_converted_model.tflite", "wb").write(tflite_model)



**Common VAE Implementation**

trained and ready for export


In [ ]:
#ref https://github.com/s-omranpour/X-VAE-keras/blob/e5ba984b5eff58f922db4442102a987619cecdc2/VAE/VAE.py
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras import backend as K
import numpy as np 
import tensorflow as tf

xx, yy = np.meshgrid(np.linspace(-7, 7, 100), np.linspace(-7, 7, 100))

def make_model(SIZE=28, LATENT_DIM=10, LR=1e-4, BETA=1.):
    encoder_inputs = layers.Input(shape=(SIZE, SIZE, 1), name='encoder_input')
    e = layers.Conv2D(filters=16,kernel_size=5,padding='SAME',activation='relu',strides=(2,2))(encoder_inputs)
    e = layers.BatchNormalization()(e)
    e = layers.Conv2D(filters=32,kernel_size=5,padding='SAME',activation='relu',strides=(2,2))(e)
    e = layers.BatchNormalization()(e)
    e = layers.Flatten()(e)
    z_mean = layers.Dense(LATENT_DIM, name='z_mean')(e)
    z_log_var = layers.Dense(LATENT_DIM, name='z_log_var')(e)
    encoder = k.Model(inputs=encoder_inputs, outputs=[z_mean, z_log_var], name='encoder')


    decoder_inputs = layers.Input(shape=(LATENT_DIM,), name='decoder_input')
    d = layers.Dense(units=7*7*4,activation='relu')(decoder_inputs)
    d = layers.Reshape((7,7,4))(d)
    d = layers.Conv2DTranspose(filters=16,kernel_size=4,strides=(2, 2), padding="SAME", activation='relu')(d)
    d = layers.Conv2DTranspose(filters=32,kernel_size=4,strides=(2, 2), padding="SAME", activation='relu')(d)
    decoded = layers.Conv2DTranspose(filters=1, kernel_size=3,strides=(1, 1), padding="SAME")(d)
    decoder = k.Model(inputs=decoder_inputs, outputs=decoded, name='decoder')


    def sample(inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    sampler = layers.Lambda(sample)
    z = sampler([z_mean, z_log_var])
    vae = k.Model(inputs=encoder_inputs, outputs=decoder(z), name='vae')

    def compute_kernel(x, y):
        x_size = tf.shape(x)[0]
        y_size = tf.shape(y)[0]
        dim = tf.shape(x)[1]
        tiled_x = tf.tile(tf.reshape(x, [x_size, 1, dim]), [1, y_size, 1])
        tiled_y = tf.tile(tf.reshape(y, [1, y_size, dim]), [x_size, 1, 1])
        return tf.exp(-tf.reduce_mean(tf.square(tiled_x - tiled_y), axis=2) / tf.cast(dim, tf.float32))


    def compute_mmd(x, y):
        x_kernel = compute_kernel(x, x)
        y_kernel = compute_kernel(y, y)
        xy_kernel = compute_kernel(x, y)
        return tf.reduce_mean(x_kernel) + tf.reduce_mean(y_kernel) - 2 * tf.reduce_mean(xy_kernel)

    true_samples = tf.random.normal(shape=tf.shape(z))
    loss_mmd = compute_mmd(true_samples, z)
    vae.add_loss(loss_mmd*BETA)
        
    vae.compile(loss='mse', optimizer=k.optimizers.Adam(LR), metrics=['mse'])
    return encoder, decoder , vae


latent_dim=2
hidden_activation='relu'
output_activation='sigmoid'
loss=mse
optimizer='adam'
epochs=100
batch_size=32
dropout_rate=0.2
l2_regularizer=0.1
validation_size=0.1
preprocessing=True
verbose=1
random_state=None
contamination=0.1
gamma=1.0
capacity=0.0
X=xx
n_samples_=X.shape[0]
n_features_ =X.shape[1]
def sampling( args):
        """Reparametrisation by sampling from Gaussian, N(0,I)
        To sample from epsilon = Norm(0,I) instead of from likelihood Q(z|X)
        with latent variables z: z = z_mean + sqrt(var) * epsilon

        Parameters
        ----------
        args : tensor
            Mean and log of variance of Q(z|X).
    
        Returns
        -------
        z : tensor
            Sampled latent variable.
        """

        z_mean, z_log = args
        batch = K.shape(z_mean)[0]  # batch size
        dim = K.int_shape(z_mean)[1]  # latent dimension
        epsilon = K.random_normal(shape=(batch, dim))  # mean=0, std=1.0

        return z_mean + K.exp(0.5 * z_log) * epsilon
def vae_loss(inputs, outputs, z_mean, z_log):
    """ Loss = Recreation loss + Kullback-Leibler loss
    for probability function divergence (ELBO).
    gamma > 1 and capacity != 0 for beta-VAE
    """

    reconstruction_loss = loss(inputs, outputs)
    reconstruction_loss *=n_features_
    kl_loss = 1 + z_log - K.square(z_mean) - K.exp(z_log)
    kl_loss = -0.5 * K.sum(kl_loss, axis=-1)
    kl_loss = gamma * K.abs(kl_loss - capacity)

    return K.mean(reconstruction_loss + kl_loss)
def make_modelv(): 
        inputs = Input(shape=(n_features_,))
        # Input layer
        layer = Dense(n_features_, activation=hidden_activation)(
            inputs)
        # Hidden layers
        for neurons in [16,1]:
            layer = Dense(neurons, activation=hidden_activation,
                          activity_regularizer=l2(l2_regularizer))(layer)
            layer = Dropout(dropout_rate)(layer)
        # Create mu and sigma of latent variables
        z_mean = Dense(latent_dim)(layer)
        z_log = Dense(latent_dim)(layer)
        # Use parametrisation sampling
        z = Lambda(sampling, output_shape=(latent_dim,))(
            [z_mean, z_log])
        # Instantiate encoder
        encoder = Model(inputs, [z_mean, z_log, z])
        if verbose >= 1:
            encoder.summary()

        # Build Decoder
        latent_inputs = Input(shape=(latent_dim,))
        # Latent input layer
        layer = Dense(latent_dim, activation=hidden_activation)(
            latent_inputs)
        # Hidden layers
        for neurons in [1,16]:
            layer = Dense(neurons, activation=hidden_activation)(layer)
            layer = Dropout(dropout_rate)(layer)
        # Output layer
        outputs = Dense(n_features_, activation=output_activation)(
            layer)
        # Instatiate decoder
        decoder = Model(latent_inputs, outputs)
        if verbose >= 1:
            decoder.summary()
        # Generate outputs
        outputs = decoder(encoder(inputs)[2])

        # Instantiate VAE
        vae = Model(inputs, outputs)
        vae.add_loss(vae_loss(inputs, outputs, z_mean, z_log))
        vae.compile(optimizer=optimizer)
        return vae 

#converting vae to a tf lite interpreter 
import tensorflow as tf
#_,_,
model = make_modelv()#AutoEncoder() #VAE([1,16],[16,1],epochs=10) 
#X_train, y_train = generate_data(behaviour='new', n_features=300, train_only=True)



'''
(train_images, y_train), (test_images, y_test) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape((28,1)).astype('float32')
test_images = test_images.reshape((28,1)).astype('float32')
train_images /= 255.
test_images /= 255.
'''
n_samples = 200
outliers_fraction = 0.25
clusters_separation = [0]

# Compare given detectors under given settings
# Initialize the data

n_inliers = int((1. - outliers_fraction) * n_samples)
n_outliers = int(outliers_fraction * n_samples)
ground_truth = np.zeros(n_samples, dtype=int)
ground_truth[-n_outliers:] = 1
model.fit(xx, xx, epochs=1, batch_size=128, verbose=0)#X_train)  

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 100)          10100       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 16)           1616        dense[0][0]                      
__________________________________________________________________________________________________
dropout (Dropout)               (None, 16)           0           dense_1[0][0]                    
______________________________________________________________________________________________

common vae export to tflite 

Input: Model 

Output: tflite file

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.allow_custom_ops = False
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                        tf.lite.OpsSet.SELECT_TF_OPS]
converter.target_spec.supported_ops=[  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
                                      tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)



INFO:tensorflow:Assets written to: /tmp/tmpw3a65zmw/assets


INFO:tensorflow:Assets written to: /tmp/tmpw3a65zmw/assets


59232

**Running tflite model**

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="pyod_converted_model.tflite")#converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)


[[0.3521298  0.2926765  0.32065654 0.3306433  0.29287452 0.3065982
  0.3014711  0.3048416  0.32357097 0.318429   0.3296155  0.31570414
  0.29687485 0.32240692 0.3058303  0.3303503  0.30831164 0.3347178
  0.34102196 0.33371425 0.3509057  0.29285443 0.29003552 0.3483403
  0.3421046  0.32886195 0.29342467 0.3275562  0.3182295  0.32856604
  0.29579797 0.30743098 0.3033597  0.3252707  0.3083835  0.31825036
  0.3012674  0.34630418 0.31980848 0.29922867 0.34561753 0.33646673
  0.34095085 0.32214624 0.32346812 0.29671413 0.3478047  0.34523755
  0.3438179  0.32386684 0.31123585 0.3102281  0.3391952  0.31694394
  0.35324875 0.3415819  0.31023216 0.30788547 0.32678473 0.3077941
  0.30900517 0.325284   0.34328794 0.33139187 0.33744705 0.34429824
  0.2990842  0.3357215  0.33033252 0.30905956 0.3467026  0.32437938
  0.3146236  0.34470195 0.32060835 0.30635837 0.35185832 0.34855193
  0.35217303 0.31467423 0.30161166 0.32443106 0.34401575 0.31414723
  0.3387174  0.32425067 0.30144906 0.3042014  0.3270

In [ ]:
!ls

converted_model.tflite	model.h5  pyod_converted_model.tflite  sample_data
